In [7]:
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
import requests
import re

In [2]:
class Batchurls():

  def __init__(self,*pages):
    templete = 'https://answers.justia.com/questions/answered/california?page='
    self.docs = [templete + str(i) for i in range(pages[0],pages[1]+1)]

    f = lambda x: BeautifulSoup(self.get_html(x))
    self.docs = list(map(f,self.docs))

  def get_html(self, url):
    x = requests.get(url)
    return x.text

  def update_doc(self):
    f = lambda x: BeautifulSoup(self.get_html(x))
    self.docs = list(map(f,self.postlinks))

  def get_qlink(self):
    self.postlinks = []
    templete = 'https://answers.justia.com'

    for doc in self.docs:
      for s in doc.find_all('strong'):
          if re.match('(Q:)',str(s.next_element)):
            link = templete + str(s.find('a')['href'])
            self.postlinks.append(link)

In [3]:
urls = Batchurls(2,3)

In [4]:
urls.get_qlink()

In [5]:
urls.update_doc()

In [6]:
urls.docs[0]

<!DOCTYPE html>
<!--[if IE 7]>
<html xml:lang="en" lang="en" dir="ltr" class="lt-ie10 lt-ie9 lt-ie8">
<![endif]--><!--[if IE 8]>
<html xml:lang="en" lang="en" dir="ltr" class="lt-ie10 lt-ie9">
<![endif]--><!--[if IE 9]>
<html xml:lang="en" lang="en" dir="ltr" class="lt-ie10 ie9">
<![endif]--><!--[if gt IE 9]>
<!--><html dir="ltr" lang="en" xml:lang="en">
<!--<![endif]-->
<head prefix="og: http://ogp.me/ns#">
<meta charset="utf-8"/>
<title>Is it proper for seller's agent to tell buyer's agent seller's financial information, e.g., no mortage? - California Real Estate Law Questions &amp; Answers - Justia Ask a Lawyer</title>
<link href="https://justatic.com/v/20230615094636/shared/css/core.css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://justatic.com/v/20230615094636/shared/css/answers.css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://justatic.com/v/20230615094636/shared/images/justia/jbutton_flat_144.svg" rel="icon" type="image/svg+xml"/>
<lin